In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
# import numpy as np

In [3]:
database_type = "/training"

#### Interpolating Data across patient timeseries

In [4]:
finalMerge = pd.read_csv('../eICU' + database_type + '/finalMerge.csv')
finalMerge = finalMerge.sort_values(by=['patientunitstayid', 'observationoffset'])
finalMerge

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
1648633,141764,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.003003,NaN
65,141764,4.0,NaN,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,141764,9.0,NaN,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,141764,14.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,141764,19.0,NaN,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1634649,3353113,2671.0,NaN,87.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
1634633,3353113,2676.0,NaN,85.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
1634665,3353113,2681.0,NaN,84.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN
1634621,3353113,2686.0,NaN,84.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
idList = sorted(set(list(finalMerge['patientunitstayid'])))
cols = ['temperature', 'heartrate', 'respiration', 'systemicsystolic', 'creatinine', 'wbcx1000', 'lactate', 'urineoutputbyweight']
finalData = pd.DataFrame()

for id in idList:
    df = finalMerge[finalMerge['patientunitstayid'] == id]
    sepsisDiagnosisTime = df[df['diagnosis'] == 1].observationoffset
    countNull = df[cols].isnull().sum().sum()
    percentOfNull = (countNull / (len(df.index) * 8)) * 100
    # print(percentOfNull)

    if percentOfNull < 80:
        if len(sepsisDiagnosisTime) != 0:
            sepsisDiagnosisTimeDict = sepsisDiagnosisTime.to_dict()
            sepsisDiagnosisTime = sepsisDiagnosisTimeDict[list(sepsisDiagnosisTimeDict.keys())[0]]
            df = df[df['observationoffset'] <= sepsisDiagnosisTime]
            df.loc[:, 'temperature'] = df['temperature'].interpolate(method='linear', limit_direction='both')
            df.loc[:, 'heartrate'] = df['heartrate'].interpolate(method='linear', limit_direction='both')
            df.loc[:, 'respiration'] = df['respiration'].interpolate(method='linear', limit_direction='both')
            df.loc[:, 'systemicsystolic'] = df['systemicsystolic'].interpolate(method='linear', limit_direction='both')
            df.loc[:, 'creatinine'] = df['creatinine'].interpolate(method='linear', limit_direction='both')
            df.loc[:, 'wbcx1000'] = df['wbcx1000'].interpolate(method='linear', limit_direction='both')
            df.loc[:, 'lactate'] = df['lactate'].interpolate(method='linear', limit_direction='both')
            df.loc[:, 'urineoutputbyweight'] = df['urineoutputbyweight'].interpolate(method='linear', limit_direction='both')
            df.loc[:, 'diagnosis'] = df['diagnosis'].interpolate(method='linear')
            df['diagnosis'].fillna('0', inplace=True)
        finalData = finalData.append(df)

finalData

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
628,143870,7.0,NaN,44.0,86.0,111.0,NaN,NaN,NaN,NaN,NaN
1677168,143870,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
574,143870,12.0,NaN,42.0,75.0,114.0,NaN,NaN,NaN,NaN,NaN
543,143870,17.0,NaN,41.0,78.0,113.0,NaN,NaN,NaN,NaN,NaN
580,143870,22.0,NaN,41.0,73.0,113.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1634649,3353113,2671.0,NaN,87.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
1634633,3353113,2676.0,NaN,85.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
1634665,3353113,2681.0,NaN,84.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN
1634621,3353113,2686.0,NaN,84.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN


#### Setting single missing cols to default human values

* temperature - 37
* heartrate - 80
* respiration - 20
* systemicsystolic - 120
* creatinine - 0.8
* wbcx1000 - 10
* lactate - 1.5
* urineoutputbyweight - 1.5

In [6]:
finalData['temperature'] = finalData['temperature'].fillna(37)
finalData['heartrate'] = finalData['heartrate'].fillna(80)
finalData['respiration'] = finalData['respiration'].fillna(20)
finalData['systemicsystolic'] = finalData['systemicsystolic'].fillna(120)
finalData['creatinine'] = finalData['creatinine'].fillna(0.8)
finalData['wbcx1000'] = finalData['wbcx1000'].fillna(10)
finalData['lactate'] = finalData['lactate'].fillna(1.5)
finalData['urineoutputbyweight'] = finalData['urineoutputbyweight'].fillna(1.5)

In [7]:
finalData[cols].isnull().sum().sum()

0

In [8]:
finalData.sort_values(['patientunitstayid','observationoffset'])

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
628,143870,7.0,37.0,44.0,86.0,111.0,0.8,10.0,1.5,1.5,NaN
1677168,143870,10.0,37.0,80.0,20.0,120.0,0.8,10.0,1.5,1.5,0
574,143870,12.0,37.0,42.0,75.0,114.0,0.8,10.0,1.5,1.5,NaN
543,143870,17.0,37.0,41.0,78.0,113.0,0.8,10.0,1.5,1.5,NaN
580,143870,22.0,37.0,41.0,73.0,113.0,0.8,10.0,1.5,1.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1634649,3353113,2671.0,37.0,87.0,27.0,120.0,0.8,10.0,1.5,1.5,NaN
1634633,3353113,2676.0,37.0,85.0,27.0,120.0,0.8,10.0,1.5,1.5,NaN
1634665,3353113,2681.0,37.0,84.0,26.0,120.0,0.8,10.0,1.5,1.5,NaN
1634621,3353113,2686.0,37.0,84.0,25.0,120.0,0.8,10.0,1.5,1.5,NaN


In [9]:
path = "../eICU/training/"
finalData.to_csv(path + "finalData.csv", sep=',', index=False, encoding='utf-8')
